# TP1- Revision sobre los SGBD relacionales

Para revisar algunos conceptos de los SGBD relacionales (en particular el modelo relacional y el lenguaje SQL), vamos a crear una pequeña Base de Datos relacional desde un programa en Python y a partir de datos que se encuentran en varios archivos textos.

La base de datos que queremos construir está representada por el modelo Entidad-Asociación siguiente:

<img src="modeloEA-1.png">

## 1. Transformación del Modelo Entidad-Asociación hacia Modelo Relacional

Proponer un modelo relacional que corresponde al modelo Entidad-Asociación.

mediosprensa (PKnombre, dueño, nfollowers)
noticias (PKID, FKnombre, titulo, texto, fecha)

## 2. Crear la estructura de la base de datos desde un programa Python

### 2.1 Conectarse a MySQL desde Python

In [69]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='root',
                              host='127.0.0.1')
DB_NAME = 'TP1'
#cnx.close()

In [33]:
cnx.close()

### 2.2 Crear las tablas

In [ ]:
from mysql.connector import errorcode



TABLES = {}

TABLES['mediosprensa'] = (
    "CREATE TABLE `mediosprensa` ("
    "  `PKnombre` varchar(50) NOT NULL,"
    "  `dueno` varchar(50) NOT NULL,"
    "  `nfollowers` int NOT NULL,"
    "  PRIMARY KEY (`PKnombre`)"
    ")" )




In [35]:
TABLES = {}
TABLES['noticias'] = (
    "CREATE TABLE `noticias` ("
    "  `PKID` int NOT NULL AUTO_INCREMENT,"
    "  `FKnombre` varchar(50) NOT NULL,"
    "  `titulo` text(100) NOT NULL,"
    "  `texto` text(10000) NOT NULL,"
    "  `fecha` date NOT NULL,"
    "  PRIMARY KEY (`PKID`),"
    "  CONSTRAINT `nombre` FOREIGN KEY (`FKnombre`) "
    "     REFERENCES `mediosprensa` (`PKnombre`) ON DELETE CASCADE"
    ")")

In [70]:
cursor = cnx.cursor()
#cursor.execute("CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
cnx.database = DB_NAME


TABLES

In [37]:
for name, ddl in TABLES.items():
    #try:
        print("Creating table {}: ".format(name), end='')
        cursor.execute(ddl)
    #except mysql.connector.Error as err:
    #    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
    #        print("already exists.")
    #    else:
    #        print(err.msg)
    #else:
    #    print("OK")


Creating table noticias: 

# 3. Insertar los datos en la base de datos

## 3.1 Leer los archivos textos

In [ ]:
import csv
with open('medios.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        print(row['CódigoMedio'], row['PropiedadLegal'], row['FollowersFinAño'])

fichero = open('noticias.txt', 'r' ) 
for linea in fichero:
    campos=linea.split("\t")
    for i in campos:
        print(i)

## 3.2 Leer los archivos textos y insertar los datos

In [71]:
cursor.execute("commit;")

In [ ]:
import csv
with open('medios.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        #print(row['CódigoMedio'], row['PropiedadLegal'], row['FollowersFinAño'])
        add_mediosprensa = "INSERT INTO mediosprensa (PKnombre, dueno, nfollowers) VALUES ('"+row['CódigoMedio']+"','"+row['PropiedadLegal']+"',"+row['FollowersFinAño']+");"               
        print(add_mediosprensa)
        cursor.execute(add_mediosprensa)

In [66]:
import csv, operator
archivo = open("noticias.txt","r")
a=0
fecha=""
nombre=""
texto=""
titulo=""
for linea in archivo.readlines():
    if(a==0):
        fecha=linea.replace("\n","")
        a+=1
    elif(a==1):
        nombre=linea.replace(" ","").replace("\n","")[0:29]
        a+=1
    elif(a==2):
        titulo=linea
        a+=1
    elif(a==3 and linea[0]!="-" and linea[1]!="-" and linea[2]!="-"):
        texto=texto+linea
    else:
        Query="insert into noticias(fecha,FKnombre,titulo,texto) values('"+fecha+"','"+nombre+"','"+titulo+"','"+texto+"');"
        print(Query);
        cursor.execute(Query)
        a=0
        texto=""
archivo.close()


insert into noticias(fecha,FKnombre,titulo,texto) values('2017-02-28 23:09:06','24HorasTVN','Casa misteriosa de Puerto Montt se convierte en atracción turística - Casa misteriosa de Puerto Montt se convierte en atracción turística - Redes Sociales - 24horas
','Masiva concurrencia a la misteriosa casa de Puerto Montt donde se registrarían supuestos sucesos paranormales. Masiva concurrencia a la misteriosa casa de Puerto Montt donde se registrarían supuestos sucesos paranormales.
');
insert into noticias(fecha,FKnombre,titulo,texto) values('2017-02-09 18:30:02','24HorasTVN','Bebé de sólo tres meses habría caído desde escaleras de tienda en Puerto Montt - Nacional - 24horas
','El lactante fue trasladado hasta el hospital Regional de Puerto Montt. Se desconoce su estado de salud.  Un bebé de sólo tres meses habría caído desde las escaleras de la tienda comercial Dolly S.A, de Puerto Montt. De acuerdo a la información entregada por radio Biobío, la víctima fue trasladada de urgencia al hosp

DatabaseError: 1366 (HY000): Incorrect string value: '\xF0\x9F\x90\xAC\xF0\x9F...' for column 'texto' at row 1

# 4. Consultar la base de datos

Listar los medios de prensa en Chile ordenado según su número de followers en Twitter.

In [50]:
cursor.execute("SELECT PKnombre,nfollowers FROM mediosprensa ORDER BY nfollowers DESC")


In [67]:
cursor.execute("SELECT COUNT(DISTINCT dueno) FROM mediosprensa")

Contar cuantes medios de prensa posee cada dueño.

In [74]:
cursor.execute("SELECT dueno, COUNT(*) as PKnombre FROM mediosprensa GROUP BY dueno")

Mostrar los últimos 10 títulos de noticias del medio "La Tercera"

In [ ]:
cursor.execute("SELECT titulo FROM noticias WHERE FKnombre = 'la tercera' LIMIT 10")

Mostrar los nombre de medios y los títulos en los cuales aparacen la palabra "Valdivia".

In [ ]:
cursor.execute("SELECT FKnombre, titulo FROM noticias WHERE texto LIKE 'Valdivia'")

Por cada medio, mostrar el número de títulos en los cuales aparecen la palabra "Puerto Montt".

In [ ]:
cursor.execute("SELECT COUT(titulo) FROM noticias WHERE titulo LIKE 'la tercera'")